In [34]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, pipeline
from sklearn import preprocessing
from tqdm import tqdm

In [2]:
#Para ajudar na análise dos dados
from sklearn.metrics import (
                    classification_report, 
                    confusion_matrix,
                    roc_curve,
                    auc,
                    accuracy_score,
                    precision_score,
                    recall_score,
                    f1_score,
                    matthews_corrcoef, #PHI
                    roc_auc_score,
                    brier_score_loss,
                    mean_squared_error,
                    mean_absolute_error,
                    median_absolute_error
                    )

In [128]:
fileResultPah = "C:\\Users\\limajos\\Dropbox\\clima_tech_artigo\\resultado_gpt_model5.csv"

In [129]:
result = 'TN'+'\t'+'TP'+'\t'+'FN'+'\t'+'FP'+'\t'+'0'+'\t'+'1'+'\t'+'accuracy'+'\t'+'precision_micro'+'\t'+'precision_macro'+'\t'+'precision_weighted'+'\t'+'precision_binary'+'\t'+'recall_micro'+'\t'+'recall_macro'+'\t'+'recall_weighted'+'\t'+'recall_binary'+'\t'+'f1Score_micro'+'\t'+'f1Score_macro'+'\t'+'f1Score_weighted'+'\t'+'f1Score_binary'+'\t'+'phi'+'\t'+'meanSquared_error'+'\t'+'meanAbsolute_error'+'\t'+'medianAbsolute_error'+'\n'

#Apaga o arquivo anterior que tenha o mesmo nome
f = open(fileResultPah, "w")
f.write(result)
f.close()


In [92]:
def salvar_mostrar_resultado(y_true, y_pred):
    
    classificationReport = classification_report(y_true, y_pred)
    
    classificationReportSplit = classificationReport.split() 
    
    confusionMatrix = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision_micro = precision_score(y_true, y_pred, average='micro')
    precision_macro = precision_score(y_true, y_pred, average='macro')
    precision_weighted = precision_score(y_true, y_pred, average='weighted')
    precision_binary = precision_score(y_true, y_pred, average='binary')
    recall_micro = recall_score(y_true, y_pred, average='micro')
    recall_macro = recall_score(y_true, y_pred, average='macro')
    recall_weighted = recall_score(y_true, y_pred, average='weighted')
    recall_binary = recall_score(y_true, y_pred, average='binary')
    f1Score_micro = f1_score(y_true, y_pred, average='micro')
    f1Score_macro = f1_score(y_true, y_pred, average='macro')
    f1Score_weighted = f1_score(y_true, y_pred, average='weighted')
    f1Score_binary = f1_score(y_true, y_pred, average='binary')
    phi = matthews_corrcoef(y_true, y_pred)
    
    meanSquared_error = mean_squared_error(y_true, y_pred)
    meanAbsolute_error = mean_absolute_error(y_true, y_pred)
    medianAbsolute_error = median_absolute_error(y_true, y_pred)
    
    matrixConfusao = str(confusionMatrix[0][0])+'\t'+str(confusionMatrix[1][1])+'\t'+str(confusionMatrix[1][0])+'\t'+str(confusionMatrix[0][1])
    precisao = str(precision_micro)+'\t'+str(precision_macro)+'\t'+str(precision_weighted)+'\t'+str(precision_binary)
    recall = str(recall_micro)+'\t'+str(recall_macro)+'\t'+str(recall_weighted)+'\t'+str(recall_binary)
    f1Score = str(f1Score_micro)+'\t'+str(f1Score_macro)+'\t'+str(f1Score_weighted)+'\t'+str(f1Score_binary)
    

    result = matrixConfusao+'\t'+str(classificationReportSplit[5])+'\t'+str(classificationReportSplit[10])+'\t'+str(accuracy)+'\t'+precisao+'\t'+recall+'\t'+f1Score+'\t'+str(phi)+'\t'+str(meanSquared_error)+'\t'+str(meanAbsolute_error)+'\t'+str(medianAbsolute_error)+'\t'+str(y_true)+'\t'+str(y_pred)+'\n'
    
    result = result.replace('.',',')

    f = open(fileResultPah, "a")
    f.write(result)
    f.close()

    print(result)  
    print(confusionMatrix)  
    print(classificationReport)

In [11]:
MAX_LENGHT = 512
TRAIN_RATIO = 0.7
VAL_RATIO = 0.2 
TEST_RATIO = 0.1

BATCH_SIZE = 16

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

In [8]:
df = pd.read_excel("./dataset_concat_v3_.xlsx")

In [9]:
dfTemp = df

In [10]:
dfTemp

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49739,50033,NaN,"Absolutamente perfeito, sem falhas!",positivo
49740,50035,NaN,"Experiência excelente, sem nenhum defeito!",positivo
49741,50037,NaN,Sem problemas detectados!,positivo
49742,50041,NaN,"Nada a reclamar, tudo perfeito!",positivo


In [11]:
# Lista de valores válidos para a coluna 'sentiment'
valores_validos = ['pos', 'neg']
#valores_validos = ['pos', 'neg', 'positivo', 'negativo']

# Filtrar o DataFrame mantendo apenas as linhas com valores válidos na coluna 'sentiment'
df_filtrado = df[df['sentiment'].isin(valores_validos)]

df = df_filtrado

df['sentiment'].value_counts()

sentiment
neg    24763
pos    24678
Name: count, dtype: int64

In [12]:
df

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49436,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49437,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49438,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49439,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [13]:
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [14]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [13]:
df['text_pt'][100]

'Imagine cada clichê estereotipado e exagerado de todos os filmes e programas de TV nas ruas de Brooklyn entre 1930 e 1980. Preencha-o com um elenco de caricaturas intercambiáveis \u200b\u200bem vez de personagens reais. Acrescente uma mistura de músicas de "período" e guitarras elétricas lamentáveis \u200b\u200bdurante as cenas "estrondosas". Em seguida, passe o tempo tentando descobrir ou importar qual dos Deuces vai ser morto no estrondo final anticlímax.Eu vou dar este filme aponta para não ser apenas mais uma comédia romântica, teen slasher, filme de ação explosivo, comédia de sexo adolescente, infantil musical, ou veículo de indicação ao Oscar. Mas trazer algo novo ou interessante para o gênero da tragédia das gangues de rua pode ter sido bom.'

In [15]:
# 'df_tokenized' será um dicionário com as keys ['input_ids', 'token_type_ids', 'attention_mask']
# 'input_ids' -> instâncias tokenizadas
# 'token_type_ids' -> mascara usada em tarefas de classificação de frases em pares (será descartada nesta task)
# 'attention_mask' -> mascara de atenção que destaca para o modelo os tokens de padding [PAD]
df_tokenized = tokenizer.batch_encode_plus(
    df['text_pt'], 
    return_tensors='pt', 
    padding=True, 
    truncation=True,
    max_length=MAX_LENGHT
)

In [16]:
df_tokenized_temp = df_tokenized

In [17]:
print(df_tokenized['input_ids'].shape, df_tokenized['attention_mask'].shape)

torch.Size([49441, 512]) torch.Size([49441, 512])


In [18]:
# [0, DATASET_LEN, MAX_LENGTH] = input_ids
# [1, DATASET_LEN, MAX_LENGTH] = attention_mask
X = torch.stack((df_tokenized['input_ids'], df_tokenized['attention_mask']), dim=0)

df['sentiment'] = df['sentiment'].apply(lambda x: 0 if (x == 'neg' or x=='negativo') else 1)
y = torch.Tensor(df['sentiment'].to_numpy())

C:\Users\limajos\AppData\Local\Temp\ipykernel_18512\45488885.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].apply(lambda x: 0 if (x == 'neg' or x=='negativo') else 1)


In [19]:
df['sentiment'].value_counts()

sentiment
0    24763
1    24678
Name: count, dtype: int64

In [18]:
class TextDataset(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.X = self.X.to(device)
        self.y = y
        self.y = self.y.to(device)
        
        self.len = len(y)
    
    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X[:, idx], self.y[idx]

In [21]:
dataset = TextDataset(X, y)

num_train_instances = np.int32(np.round(dataset.len * TRAIN_RATIO))
num_val_instances = np.int32(np.round(dataset.len * VAL_RATIO))
num_test_instances = np.int32(np.round(dataset.len * TEST_RATIO))
print(f"Treino: {num_train_instances}, Val: {num_val_instances}, Teste: {num_test_instances}")

train_split, val_split, test_split = torch.utils.data.random_split(dataset, [num_train_instances, num_val_instances, num_test_instances])

train_loader = torch.utils.data.DataLoader(train_split, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_split, batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_split, batch_size=BATCH_SIZE, shuffle=True)

Treino: 34609, Val: 9888, Teste: 4944


In [23]:
epochs = 11#500
steps_per_epochs = 200#200
epochs_validation_samples = 50#50

model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased').to(device)

for param in model.base_model.parameters():
    param.requires_grad = False
    
loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

acc_calc = lambda output, labels : (labels == output.argmax(axis=1)).sum()

scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
epoch_metadata = []

for i in range(epochs):
    
    num_train_examples = 0
    num_val_examples = 0
    
    train_hits = 0
    val_hits = 0
    
    train_bar = tqdm(total=steps_per_epochs, desc=f"Train", unit="steps", position=0, leave=True)
    val_bar = tqdm(total=epochs_validation_samples, desc=f"VAL", unit="samples", position=0, leave=True)
    
    for batch_number, (features, labels) in enumerate(train_loader):
        
        train_running_loss = 0
        
        model.train()
        
        input_ids, input_masks = features[:, 0, :], features[:, 1, :]
        
        loss, logits = model(input_ids, input_masks, labels=labels.long()).to_tuple()
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        train_running_loss += loss.item()
        
        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        train_hits += acc_calc(softmax_predictions, labels)
        
        train_bar.update(1)
        
        num_train_examples += features.shape[0]
        
        scheduler.step()
        
        if (batch_number + 1) % steps_per_epochs == 0:
            train_bar.close()
            break
            
    for batch_number, (features, labels) in enumerate(val_loader):
        with torch.no_grad():
            val_running_loss = 0

            model.eval()

            input_ids, input_masks = features[:, 0, :], features[:, 1, :]
            loss, logits = model(input_ids, input_masks, labels=labels.long()).to_tuple()

            val_running_loss += loss.item()

            softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
            val_hits += acc_calc(softmax_predictions, labels)

            num_val_examples += features.shape[0]

            val_bar.update(1)

            scheduler.step()

            if (batch_number + 1) % epochs_validation_samples == 0:
                val_bar.close()
                break
    train_acc = torch.true_divide(train_hits, num_train_examples)
    val_acc = torch.true_divide(val_hits, num_val_examples)
    
    print(f"EPOCH SUMMARY - {i + 1} \t Train loss: {train_running_loss} \t Train Acc: {train_acc} \t Val loss: {val_running_loss} \t Val acc: {val_acc}")
    

VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:12:06<00:00, 86.52s/samples]


EPOCH SUMMARY - 1 	 Train loss: 0.5635520219802856 	 Train Acc: 0.6543750166893005 	 Val loss: 0.6135963797569275 	 Val acc: 0.706250011920929


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:11:00<00:00, 85.21s/samples]


EPOCH SUMMARY - 2 	 Train loss: 0.6375169157981873 	 Train Acc: 0.7149999737739563 	 Val loss: 0.6026898622512817 	 Val acc: 0.7200000286102295


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:10:43<00:00, 84.86s/samples]


...

EPOCH SUMMARY - 9 	 Train loss: 0.6366626024246216 	 Train Acc: 0.7696874737739563 	 Val loss: 0.48261117935180664 	 Val acc: 0.8050000071525574


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:17:59<00:00, 93.59s/samples]


EPOCH SUMMARY - 10 	 Train loss: 0.5461646914482117 	 Train Acc: 0.7909374833106995 	 Val loss: 0.3902184069156647 	 Val acc: 0.8025000095367432


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:15:15<00:00, 90.31s/samples]

EPOCH SUMMARY - 11 	 Train loss: 0.5236168503761292 	 Train Acc: 0.7706249952316284 	 Val loss: 0.48774901032447815 	 Val acc: 0.8025000095367432


VAL: 100%|█████████████████████████████████████████████████████████████████████| 50/50 [1:34:04<00:00, 112.89s/samples]


EPOCH SUMMARY - 1 	 Train loss: 0.6464643478393555 	 Train Acc: 0.6621875166893005 	 Val loss: 0.6763045191764832 	 Val acc: 0.7074999809265137


VAL: 100%|█████████████████████████████████████████████████████████████████████| 50/50 [1:27:23<00:00, 104.86s/samples]


EPOCH SUMMARY - 2 	 Train loss: 0.5170610547065735 	 Train Acc: 0.7212499976158142 	 Val loss: 0.49533841013908386 	 Val acc: 0.7487499713897705


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:11:29<00:00, 85.78s/samples]


...

EPOCH SUMMARY - 21 	 Train loss: 0.4608862102031708 	 Train Acc: 0.7928125262260437 	 Val loss: 0.32558324933052063 	 Val acc: 0.8125


VAL: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [1:10:48<00:00, 84.96s/samples]


EPOCH SUMMARY - 22 	 Train loss: 0.6076433658599854 	 Train Acc: 0.7853124737739563 	 Val loss: 0.45986542105674744 	 Val acc: 0.7912499904632568


Train:  72%|██████████████████████████████████████████████████▍                   | 144/200 [42:07<21:21, 22.88s/steps]

KeyboardInterrupt: 

In [25]:
model.save_pretrained(f"./modelimdb")

In [26]:
num_test_examples = 0

test_hits = 0

test_running_loss = 0

for batch_number, (features, labels) in enumerate(test_loader):
        with torch.no_grad():
            test_running_loss = 0

            model.eval()

            input_ids, input_masks = features[:, 0, :], features[:, 1, :]
            loss, logits = model(input_ids, input_masks, labels=labels.long()).to_tuple()

            test_running_loss += loss.item()

            softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
            test_hits += acc_calc(softmax_predictions, labels)

            num_test_examples += features.shape[0]
            
            # Get predicted label
            _, predicted_label = torch.max(softmax_predictions, 1)

            # Convert predicted_label tensor to a Python list
            predicted_labels = predicted_label.tolist()

            salvar_mostrar_resultado(labels, predicted_labels)

In [27]:
test_acc = torch.true_divide(test_hits, num_test_examples)
test_acc

tensor(0.8091)

# Avalidar modelo IMDB com a base dados ChatGpt

In [24]:
# Caminho dos arquivos salvos
model_path = f"./modelimdb"
model = BertForSequenceClassification.from_pretrained(model_path).to(device)

In [7]:
dfGpt = pd.read_excel("./dataset_concat_v3_.xlsx")

In [8]:
# Lista de valores válidos para a coluna 'sentiment'
valores_validos = ['positivo', 'negativo']
#valores_validos = ['pos', 'neg', 'positivo', 'negativo']

# Filtrar o DataFrame mantendo apenas as linhas com valores válidos na coluna 'sentiment'
df_filtrado = dfGpt[dfGpt['sentiment'].isin(valores_validos)]

dfGpt = df_filtrado

dfGpt['sentiment'].value_counts()

sentiment
positivo    159
negativo    144
Name: count, dtype: int64

In [12]:
# Carregar o modelo e o tokenizador
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# 'df_tokenized' será um dicionário com as keys ['input_ids', 'token_type_ids', 'attention_mask']
# 'input_ids' -> instâncias tokenizadas
# 'token_type_ids' -> mascara usada em tarefas de classificação de frases em pares (será descartada nesta task)
# 'attention_mask' -> mascara de atenção que destaca para o modelo os tokens de padding [PAD]
dfGpt_tokenized = tokenizer.batch_encode_plus(
    dfGpt['text_pt'], 
    return_tensors='pt', 
    padding=True, 
    truncation=True,
    max_length=MAX_LENGHT
)

In [14]:
print(dfGpt_tokenized['input_ids'].shape, dfGpt_tokenized['attention_mask'].shape)

torch.Size([303, 33]) torch.Size([303, 33])


In [19]:
# [0, DATASET_LEN, MAX_LENGTH] = input_ids
# [1, DATASET_LEN, MAX_LENGTH] = attention_mask
XGpt = torch.stack((dfGpt_tokenized['input_ids'], dfGpt_tokenized['attention_mask']), dim=0)

dfGpt['sentiment'] = dfGpt['sentiment'].apply(lambda x: 0 if (x == 'neg' or x=='negativo') else 1)
yGpt = torch.Tensor(dfGpt['sentiment'].to_numpy())

In [20]:
dfGpt['sentiment'].value_counts()

sentiment
1    303
Name: count, dtype: int64

In [22]:
datasetGpt = TextDataset(XGpt, yGpt)

testGpt_loader = torch.utils.data.DataLoader(datasetGpt, batch_size=500)

In [36]:
num_test_examplesGpt = 0

test_hitsGpt = 0

test_running_lossGpt = 0

for batch_numberGpt, (featuresGpt, labelsGpt) in enumerate(testGpt_loader):
        with torch.no_grad():
            test_running_lossGpt = 0

            model.eval()

            input_ids, input_masks = featuresGpt[:, 0, :], featuresGpt[:, 1, :]
            loss, logits = model(input_ids, input_masks, labels=labelsGpt.long()).to_tuple()

            test_running_lossGpt += loss.item()

            acc_calcGpt = lambda output, labels : (labels == output.argmax(axis=1)).sum()
        
            softmax_predictionsGpt = torch.nn.functional.softmax(logits, dim=1)
            test_hitsGpt += acc_calcGpt(softmax_predictionsGpt, labelsGpt)

            num_test_examplesGpt += featuresGpt.shape[0]
            
            # Get predicted label
            _, predicted_labelGpt = torch.max(softmax_predictionsGpt, 1)

            # Convert predicted_label tensor to a Python list
            predicted_labelsGpt = predicted_labelGpt.tolist()

            salvar_mostrar_resultado(labelsGpt, predicted_labelsGpt)

0	90	213	0	0,00	1,00	0,297029702970297	0,297029702970297	0,5	1,0	1,0	0,297029702970297	0,1485148514851485	0,297029702970297	0,297029702970297	0,297029702970297	0,22900763358778625	0,4580152671755725	0,4580152671755725	0,0	0,7029702970297029	0,7029702970297029	1,0

[[  0   0]
 [213  90]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         0
         1.0       1.00      0.30      0.46       303

    accuracy                           0.30       303
   macro avg       0.50      0.15      0.23       303
weighted avg       1.00      0.30      0.46       303



C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Reca

# Gerar modelo com a base dados ChatGpt a partir do modelo IMDB

In [176]:
dfGptModel = pd.read_excel("./dataset_concat_v3_.xlsx")

In [177]:
# Lista de valores válidos para a coluna 'sentiment'
valores_validos = ['positivo', 'negativo']
#valores_validos = ['pos', 'neg', 'positivo', 'negativo']

# Filtrar o DataFrame mantendo apenas as linhas com valores válidos na coluna 'sentiment'
df_filtrado = dfGptModel[dfGptModel['sentiment'].isin(valores_validos)]

dfGptModel = df_filtrado

dfGptModel['sentiment'].value_counts()

sentiment
positivo    159
negativo    144
Name: count, dtype: int64

In [178]:
# Carregar o modelo e o tokenizador
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# 'df_tokenized' será um dicionário com as keys ['input_ids', 'token_type_ids', 'attention_mask']
# 'input_ids' -> instâncias tokenizadas
# 'token_type_ids' -> mascara usada em tarefas de classificação de frases em pares (será descartada nesta task)
# 'attention_mask' -> mascara de atenção que destaca para o modelo os tokens de padding [PAD]
dfGptModel_tokenized = tokenizer.batch_encode_plus(
    dfGptModel['text_pt'], 
    return_tensors='pt', 
    padding=True, 
    truncation=True,
    max_length=MAX_LENGHT
)

In [179]:
print(dfGptModel_tokenized['input_ids'].shape, dfGptModel_tokenized['attention_mask'].shape)

torch.Size([303, 33]) torch.Size([303, 33])


In [180]:
# [0, DATASET_LEN, MAX_LENGTH] = input_ids
# [1, DATASET_LEN, MAX_LENGTH] = attention_mask
XGptModel = torch.stack((dfGptModel_tokenized['input_ids'], dfGptModel_tokenized['attention_mask']), dim=0)

dfGptModel['sentiment'] = dfGptModel['sentiment'].apply(lambda x: 0 if (x == 'neg' or x=='negativo') else 1)
yGptModel = torch.Tensor(dfGptModel['sentiment'].to_numpy())

In [181]:
dfGptModel['sentiment'].value_counts()

sentiment
1    159
0    144
Name: count, dtype: int64

In [182]:
datasetGptModel = TextDataset(XGptModel, yGptModel)

num_trainGptModel_instances = np.int32(np.round(datasetGptModel.len * TRAIN_RATIO))
num_valGptModel_instances = np.int32(np.round(datasetGptModel.len * VAL_RATIO))
num_testGptModel_instances = np.int32(np.round(datasetGptModel.len * TEST_RATIO))
print(f"Treino: {num_trainGptModel_instances}, Val: {num_valGptModel_instances}, Teste: {num_testGptModel_instances}")

trainGptModel_split, valGptModel_split, testGptModel_split = torch.utils.data.random_split(datasetGptModel, [num_trainGptModel_instances, num_valGptModel_instances, num_testGptModel_instances])

trainGptModel_loader = torch.utils.data.DataLoader(trainGptModel_split, batch_size=BATCH_SIZE, shuffle=True)
valGptModel_loader = torch.utils.data.DataLoader(valGptModel_split, batch_size=BATCH_SIZE, shuffle=True)
#testGptModel_loader = torch.utils.data.DataLoader(testGptModel_split, batch_size=BATCH_SIZE, shuffle=True)
testGptModel_loader = torch.utils.data.DataLoader(testGptModel_split, batch_size=500)

Treino: 212, Val: 61, Teste: 30


In [183]:
epochs = 11#500
steps_per_epochs = 200#200
epochs_validation_samples = 50#50

# Caminho dos arquivos salvos
model_path = f"./modelimdb"
modelGptModel = BertForSequenceClassification.from_pretrained(model_path).to(device)

for param in modelGptModel.base_model.parameters():
    param.requires_grad = False
    
loss_func = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(modelGptModel.parameters())

acc_calc = lambda output, labels : (labels == output.argmax(axis=1)).sum()

scheduler = torch.optim.lr_scheduler.ExponentialLR(optim, 0.9997)

In [184]:
epoch_metadata = []

for i in range(epochs):
    
    num_train_examples = 0
    num_val_examples = 0
    
    train_hits = 0
    val_hits = 0
    
    train_bar = tqdm(total=steps_per_epochs, desc=f"Train", unit="steps", position=0, leave=True)
    val_bar = tqdm(total=epochs_validation_samples, desc=f"VAL", unit="samples", position=0, leave=True)
    
    for batch_number, (features, labels) in enumerate(trainGptModel_loader):
        
        train_running_loss = 0
        
        modelGptModel.train()
        
        input_ids, input_masks = features[:, 0, :], features[:, 1, :]
        
        loss, logits = modelGptModel(input_ids, input_masks, labels=labels.long()).to_tuple()
        
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        train_running_loss += loss.item()
        
        softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
        train_hits += acc_calc(softmax_predictions, labels)
        
        train_bar.update(1)
        
        num_train_examples += features.shape[0]
        
        scheduler.step()
        
        if (batch_number + 1) % steps_per_epochs == 0:
            train_bar.close()
            break
            
    for batch_number, (features, labels) in enumerate(valGptModel_loader):
        with torch.no_grad():
            val_running_loss = 0

            modelGptModel.eval()

            input_ids, input_masks = features[:, 0, :], features[:, 1, :]
            loss, logits = modelGptModel(input_ids, input_masks, labels=labels.long()).to_tuple()

            val_running_loss += loss.item()

            softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
            val_hits += acc_calc(softmax_predictions, labels)

            num_val_examples += features.shape[0]

            val_bar.update(1)

            scheduler.step()

            if (batch_number + 1) % epochs_validation_samples == 0:
                val_bar.close()
                break
    train_acc = torch.true_divide(train_hits, num_train_examples)
    val_acc = torch.true_divide(val_hits, num_val_examples)
    
    print(f"EPOCH SUMMARY - {i + 1} \t Train loss: {train_running_loss} \t Train Acc: {train_acc} \t Val loss: {val_running_loss} \t Val acc: {val_acc}")
    

VAL:   8%|█████▊                                                                   | 4/50 [00:08<00:58,  1.27s/samples]

EPOCH SUMMARY - 1 	 Train loss: 0.3187931180000305 	 Train Acc: 0.8396226167678833 	 Val loss: 0.4272170662879944 	 Val acc: 0.868852436542511


VAL:   8%|█████▊                                                                   | 4/50 [00:09<01:08,  1.48s/samples]

EPOCH SUMMARY - 2 	 Train loss: 0.21408092975616455 	 Train Acc: 0.8301886916160583 	 Val loss: 0.45731469988822937 	 Val acc: 0.9344262480735779


VAL:   8%|█████▊                                                                   | 4/50 [00:12<01:28,  1.91s/samples]

...

EPOCH SUMMARY - 9 	 Train loss: 0.22785943746566772 	 Train Acc: 0.9433962106704712 	 Val loss: 0.19945308566093445 	 Val acc: 0.9672130942344666


VAL:   8%|█████▊                                                                   | 4/50 [00:09<01:11,  1.55s/samples]

EPOCH SUMMARY - 10 	 Train loss: 0.40130382776260376 	 Train Acc: 0.9103773832321167 	 Val loss: 0.22405889630317688 	 Val acc: 0.9672130942344666


VAL:   8%|█████▊                                                                   | 4/50 [00:10<01:14,  1.63s/samples]

EPOCH SUMMARY - 11 	 Train loss: 0.142496719956398 	 Train Acc: 0.900943398475647 	 Val loss: 0.1541433334350586 	 Val acc: 0.9672130942344666


In [185]:
#modelGptModel.save_pretrained(f"./modelimdb_chatgpt")

In [186]:
num_test_examples = 0

test_hits = 0

test_running_loss = 0

all_input_ids = []
all_labels = []

for batch_number, (features, labels) in enumerate(testGptModel_loader):
        with torch.no_grad():
            test_running_loss = 0

            modelGptModel.eval()

            input_ids, input_masks = features[:, 0, :], features[:, 1, :]
            loss, logits = modelGptModel(input_ids, input_masks, labels=labels.long()).to_tuple()

            all_input_ids += input_ids
            all_labels += labels
            
            test_running_loss += loss.item()

            softmax_predictions = torch.nn.functional.softmax(logits, dim=1)
            test_hits += acc_calc(softmax_predictions, labels)

            num_test_examples += features.shape[0]
            
            # Get predicted label
            _, predicted_label = torch.max(softmax_predictions, 1)

            # Convert predicted_label tensor to a Python list
            predicted_labels = predicted_label.tolist()

            salvar_mostrar_resultado(labels, predicted_labels)

14	14	1	1	0,93	0,93	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,9333333333333333	0,8666666666666667	0,06666666666666667	0,06666666666666667	0,0	tensor([0,, 1,, 0,, 1,, 1,, 0,, 1,, 0,, 1,, 1,, 0,, 1,, 1,, 1,, 0,, 1,, 0,, 1,,
        0,, 0,, 1,, 0,, 1,, 0,, 0,, 1,, 0,, 1,, 0,, 0,])	[0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1]

[[14  1]
 [ 1 14]]
              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93        15
         1.0       0.93      0.93      0.93        15

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



In [141]:
train_test = torch.true_divide(test_hits, num_test_examples)
print(test_hits)
print(train_test)

tensor(30)
tensor(1.)


In [142]:
for index, item in enumerate(all_input_ids):
    print(index)
    predicted_tokens = tokenizer.convert_ids_to_tokens(item)
    # Remove special tokens and convert to string
    predicted_tokens = [token for token in predicted_tokens if token not in ['[CLS]', '[SEP]', '[PAD]']]
    predicted_text = ' '.join(predicted_tokens)
    print('Texto: ', predicted_text)

0
Texto:  Excel ##ente produto , fácil de usar e muito eficaz na refrig ##eração .
1
Texto:  Não recomend ##o , apresentou defe ##itos logo nos primeiros dias .
2
Texto:  Rec ##ome ##ndo fortemente este ar - condicion ##ado , sua eficiência e facilidade de operação são impressionante ##s .
3
Texto:  Rui ##m , bar ##ul ##hen ##to e não res ##fri ##a o ambiente adequada ##mente .
4
Texto:  Nenhum problema , sem falhas !
5
Texto:  Est ##ou satisf ##eito com a eficiência e economia deste produto .
6
Texto:  Sem nenhum defe ##ito , excelente !
7
Texto:  Re ##fri ##ger ##a bem , mas é bar ##ul ##hen ##to .
8
Texto:  Ad ##qui ##ri recentemente o ar - condicion ##ado Sam ##su ##ng e estou extremamente satisf ##eito com sua capacidade de refrig ##eração eficiente e economia de energia .
9
Texto:  Infelizmente , este ar - condicion ##ado não atende ##u às expectativas , apresentando falhas frequentes e preocupa ##ntes .
10
Texto:  Est ##ou muito conte ##nte com a compra deste ar - condicion ##ad

In [123]:
allLabels = []
for batch_number, (features, labels) in enumerate(testGptModel_loader):
    allLabels += labels
print(allLabels)

[tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(0.)]


In [63]:
data = testGptModel_loader.dataset

In [99]:
#predicted_tokens = tokenizer.convert_ids_to_tokens(list(data)[0][1])
#print(data[0][0][0])
#for t in list(data):
for index, item in enumerate(data):
    print(index)
    #print(t[0][0])
    # Convert token IDs back to tokens
    predicted_tokens = tokenizer.convert_ids_to_tokens(item[0][0])
    # Remove special tokens and convert to string
    predicted_tokens = [token for token in predicted_tokens if token not in ['[CLS]', '[SEP]', '[PAD]']]
    predicted_text = ' '.join(predicted_tokens)
    print('Texto: ', predicted_text)
    #print('index: ', tokenizer.convert_ids_to_tokens(item[0][1]))

0
Texto:  Fi ##quei desa ##pon ##tado , muitas coisas er ##radas .
1
Texto:  Est ##ou sur ##pres ##o com a capacidade de refrig ##eração deste ar - condicion ##ado Sam ##su ##ng , é altamente eficiente e econômico .
2
Texto:  Dec ##ep ##cionado com a qualidade deste ar - condicion ##ado Sam ##su ##ng , desde o início apresentou defe ##itos constantes .
3
Texto:  Não encont ##rei nenhuma falha , tudo perfeito .
4
Texto:  Não encont ##rei nenhum defe ##ito , recomend ##o totalmente .
5
Texto:  Deixou a deseja ##r , não atende ##u minha ##s expectativas .
6
Texto:  Est ##ou extremamente satisf ##eito com a qualidade deste ar - condicion ##ado Sam ##su ##ng , sua eficiência e desempenho são excelentes .
7
Texto:  Não recomend ##o este ar - condicion ##ado , sua baixa dura ##bilidade e eficiência abaixo do esperado são preocupa ##ntes .
8
Texto:  Est ##ou satisf ##eito com a eficiência deste produto .
9
Texto:  Desem ##penh ##o satisfa ##tório , estou satisf ##eito .
10
Texto:  Não correspo

In [89]:
#predicted_tokens = tokenizer.convert_ids_to_tokens(list(data)[0][1])
#print(data[0][0][0])
#for t in list(data):
for index, item in enumerate(data):
    print(index)
    #print(t[0][0])
    # Convert token IDs back to tokens
    predicted_tokens = tokenizer.convert_ids_to_tokens(item[0][0])
    # Remove special tokens and convert to string
    predicted_tokens = [token for token in predicted_tokens if token not in ['[CLS]', '[SEP]', '[PAD]']]
    predicted_text = ' '.join(predicted_tokens)
    print('Texto: ', predicted_text)
    #print(tokenizer.convert_ids_to_tokens(t[0][0]))
    print('Label: ', item[0][1])

0
Texto:  Fi ##quei desa ##pon ##tado , muitas coisas er ##radas .
Label:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])
1
Texto:  Est ##ou sur ##pres ##o com a capacidade de refrig ##eração deste ar - condicion ##ado Sam ##su ##ng , é altamente eficiente e econômico .
Label:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0])
2
Texto:  Dec ##ep ##cionado com a qualidade deste ar - condicion ##ado Sam ##su ##ng , desde o início apresentou defe ##itos constantes .
Label:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 0])
3
Texto:  Não encont ##rei nenhuma falha , tudo perfeito .
Label:  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0])
4
Texto:  Não encont ##rei nenhum defe ##ito , recomend ##o totalmente .
Label:  tensor([1

In [65]:
list(data)

[(tensor([[  101, 19687, 19623,  1950,  1108,   487,   117,  1615,  4486,  2317,
            2208,   119,   102,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0],
          [    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
               1,     1,     1,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0]]),
  tensor(0.)),
 (tensor([[  101,  9009,   203,  1401,   491, 22280,   170,   123,  2839,   125,
           16373,  3917,  2166,   388,   118, 13775,   201,  3021,  2515,   833,
             117,   253,  7532, 10746,   122,  5912,   119,   102,     0,     0,
               0,     0,     0],
          [    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
               1,     1,     1,     1,     1,     1,     1,     1,     1, 

In [115]:
allLabels = []
for batch_number, (features, labels) in enumerate(testGptModel_loader):
    allLabels += labels
print(allLabels)

[tensor(1.), tensor(0.), tensor(1.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(0.), tensor(0.), tensor(0.), tensor(1.), tensor(1.), tensor(1.)]


In [61]:
testGptModel_loader

In [60]:
testGptModel_loader

In [111]:
i1, l1 = next(iter(testGptModel_loader))
print(i1.shape)

torch.Size([16, 2, 33])


In [112]:
print(l1)

tensor([0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.])


In [113]:
print(l1)

tensor([0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1.])


In [59]:
testGptModel_loader.to_excel(r'testGptModel_loader.xlsx', engine='xlsxwriter', index=False)

AttributeError: 'DataLoader' object has no attribute 'to_excel'

In [ ]:
df = pd.read_csv("./dataset_concat_v3.csv")

In [6]:
df

,id,text_en,text_pt,sentiment,Unnamed: 4,Unnamed: 5
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,NaN,NaN
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,NaN,NaN
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,NaN,NaN
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,NaN,NaN
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,NaN,NaN
...,...,...,...,...,...,...
50038,50055,NaN,"Experiência excelente, sem nenhum defeito!",positivo,NaN,NaN
50039,50056,NaN,"Não gostei, muitos problemas encontrados.",negativo,NaN,NaN
50040,50057,NaN,"Sem problemas detectados, serviço perfeito!",positivo,NaN,NaN
50041,50058,NaN,"Deixou muito a desejar, não recomendo.",negativo,NaN,NaN


In [7]:
df2 = df.drop_duplicates()

In [8]:
df2

,id,text_en,text_pt,sentiment,Unnamed: 4,Unnamed: 5
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg,NaN,NaN
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg,NaN,NaN
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg,NaN,NaN
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg,NaN,NaN
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg,NaN,NaN
...,...,...,...,...,...,...
50038,50055,NaN,"Experiência excelente, sem nenhum defeito!",positivo,NaN,NaN
50039,50056,NaN,"Não gostei, muitos problemas encontrados.",negativo,NaN,NaN
50040,50057,NaN,"Sem problemas detectados, serviço perfeito!",positivo,NaN,NaN
50041,50058,NaN,"Deixou muito a desejar, não recomendo.",negativo,NaN,NaN


In [11]:
df.to_excel(r'dataset_concat_v3.xlsx', engine='xlsxwriter', index=False)